In [1]:
# 导入TF
import tensorflow as tf
# 导入Keras相关模块
from tensorflow.keras import layers, Sequential
from tensorflow import keras

# 06. 神经网络
机器学习的最终目的是找到一组良好的参数$\theta$，使得$\theta$表示的数学模型能够很好地从训练集中学到映射关系$f_{\theta}:x \to y, \  x,y \in \mathbb{D}^{\mathrm{train}}$，从而利用训练好的$f_{\theta}(x), x \in \mathbb{D}^{\mathrm{test}}$去预测新样本。

## 6.1 感知机
感知机模型的结构如`图6.1`所示，它接受长度为$n$的一维向量$x = [x_1,x_2,\dots,x_n]$，每个输入节点通过权值为$w_i, i \in [1, n]$的连接汇集为变量$z$，即：
+ $𝑧 = w_1x_1 + w_2x_2 + \dots + w_nx_n + b$

其中$b$称为感知机的`偏置`(Bias)，一维向量$w=[w_1,w_2,\dots,w_n]$称为感知机的`权值`(Weight)，$z$称为感知机的`净活性值`(Net Activation)。

<img src="images/06_01.png" style="width:400px;"/>

上面的公式也可以写成向量形式：
+ $z = w^Tx + b$

感知机是线性模型，并不能处理线性不可分问题。通过在线性模型后添加激活函数后得到`活性值`(Activation):
+ $a = \sigma(z) = \sigma(w^Tx + b)$

其中激活函数可以是阶跃函数(Step function)、也可以是符号函数(Sign function)：

<img src="images/06_02.png" style="width:600px;"/>

添加激活函数后，感知机可以用来完成二分类任务。阶跃函数和符号函数在$z = 0$处是不连续的，其他位置导数为0，无法利用梯度下降算法进行参数优化。

为了能够让感知机模型能够从数据中间自动学习，`Frank Rosenblatt`提出了感知机的学习算法，如`算法1`所示：

<img src="images/a_06_01.png" style="width:500px;"/>

虽然感知机提出之处被寄予了良好的发展潜力，但是`Marvin Lee Minsky`和`Seymour Papert`于1969年在《Perceptrons》书中证明了以感知机为代表的线性模型不能解决`异或`(XOR)等线性不可分问题，这直接导致了当时新兴的神经网络的研究进入了低谷期。尽管感知机模型不能解决线性不可分问题，但书中也提到通过嵌套多层神经网络可以解决。

## 6.2 全连接层
感知机模型的不可导特性严重约束了它的潜力，使得它只能解决极其简单的任务。实际上，现代深度学习动辄数百万甚至上亿的参数规模，但它的核心结构与感知机并没有多大差别。它在感知机的基础上，将不连续的阶跃激活函数换成了其它平滑连续可导的激活函数，并通过堆叠多个网络层来增强网络的表达能力。

我们通过替换感知机的激活函数，同时并行堆叠多个神经元来实现多输入、多输出的网络层结构。如`图6.4`所示：

<img src="images/06_04.png" style="width:250px;"/>

第一个输出节点的输出为：
+ $o_1 = \sigma(w_11\cdot x_1 + w_21\cdot x_2 + w_31\cdot x_3 + b_1)$

第二个输出节点的输出为：
+ $o_2 = \sigma(w_12\cdot x_1 + w_22\cdot x_2 + w_32\cdot x_3 + b_1)$

输出向量为$o = [o_1, o_2]$。整个网络层可以通过矩阵关系式表达：
+ $[o_1, o_2] = [x_1\ x_2\ x_3]@\begin{bmatrix}w_{11}&w_{12}\\w_{21}&w_{22}\\w_{31}&w_{32}\\ \end{bmatrix} + [b_1\ b_2]$

向量形式表示：
+ $O = X@W + b$

由于每个输出节点与全部的输入节点相连接，这种网络层称为`全连接层`(Fully-connected Layer)，或者`稠密连接层`(Dense Layer)，$W$矩阵叫做全连接层的权值矩阵，$b$向量叫做全连接层的偏置向量。

### 6.2.1 张量方式实现
在`TensorFlow`中，要实现全连接层，只需要定义好权值张量$W$和偏置张量$b$，并利用批量矩阵相乘函数`tf.matmul()`即可完成网络层的计算。

In [2]:
# 创建`W,b`张量
x = tf.random.normal([2,784])
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))
# 线性变换
o1 = tf.matmul(x,w1) + b1
# 激活函数
o1 = tf.nn.relu(o1)

### 6.2.2 层方式实现
作为最常用的网络层之一，`TensorFlow`中有更高层、使用更方便的全连接层实现方式：`layers.Dense(units, activation)`。 通过`layer.Dense`类，只需要指定输出节点数`Units`和激活函数类型`activation`即可。

输入节点数会根据第一次运算时的输入`shape`确定，同时根据输入、输出节点数自动创建并初始化权值张量$W$和偏置张量$b$，因此在新建类`Dense`实例时，并不会立即创建权值张量$W$和偏置张量$b$，而是需要调用`build`函数或者直接进行一次前向计算，才能完成网络参数的创建。`activation`参数指定当前层的激活函数，可以为常见的激活函数或自定义激活函数，也可以指定为`None`，即无激活函数。

In [3]:
x = tf.random.normal([4,28*28])
# 创建全连接层，指定输出节点数和激活函数
fc = layers.Dense(512, activation=tf.nn.relu)
# 通过fc类实例完成一次全连接层的计算，返回输出张量
h1 = fc(x)
h1

<tf.Tensor: shape=(4, 512), dtype=float32, numpy=
array([[0.        , 0.33936763, 0.12804674, ..., 0.        , 0.25500816,
        0.        ],
       [0.79659146, 0.16415203, 0.02738798, ..., 0.        , 0.        ,
        0.63829184],
       [0.        , 0.        , 1.6140906 , ..., 0.9834901 , 0.8804554 ,
        0.        ],
       [0.        , 1.5699258 , 0.        , ..., 0.        , 1.5728927 ,
        0.        ]], dtype=float32)>

上述代码创建了一层全连接层`fc`，并指定输出节点数为512，输入的节点数在`fc(x)`计算时自动获取，并创建内部权值张量$W$和偏置张量$b$。我们可以通过类内部的成员名`kernel`和`bias`来获取权值张量$W$和偏置张量$b$对象：

In [4]:
fc.kernel # 获取Dense类的权值矩阵

<tf.Variable 'dense/kernel:0' shape=(784, 512) dtype=float32, numpy=
array([[-5.2117623e-02,  4.7515027e-02, -5.9975043e-02, ...,
         6.1699390e-02, -2.4923325e-02, -5.9274010e-02],
       [-2.2647947e-02, -3.3807483e-02,  5.2326262e-02, ...,
        -7.3583201e-03,  6.6806361e-02, -4.0036727e-02],
       [ 1.8346965e-02, -5.6436360e-03,  4.2286471e-02, ...,
        -1.6777068e-02,  2.5022767e-02,  5.3288117e-03],
       ...,
       [ 9.7975135e-06, -4.7732517e-02,  4.8210979e-02, ...,
         2.3550361e-02,  3.5267122e-02,  9.9150389e-03],
       [ 4.2524047e-02,  1.9183196e-02, -5.4806851e-02, ...,
        -2.3105964e-03,  6.1260149e-02,  6.3090995e-02],
       [ 4.4858798e-02, -2.2524446e-02, -1.2210969e-02, ...,
         3.0123860e-02, -3.9718702e-02,  3.3546530e-02]], dtype=float32)>

In [5]:
fc.bias # 获取Dense类的偏置向量

<tf.Variable 'dense/bias:0' shape=(512,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.

在优化参数时，需要获得网络的所有待优化的张量参数列表，可以通过类的`trainable_variables`来返回待优化参数列表：

In [6]:
for v in fc.trainable_variables:
    print("name: ", v.name, ", shape: ", v.shape)

name:  dense/kernel:0 , shape:  (784, 512)
name:  dense/bias:0 , shape:  (512,)


网络层除了保存了待优化张量列表`trainable_variables`，还有部分层包含了不参与梯度优化的张量，如后续介绍的`Batch Normalization`层，可以通过`non_trainable_variables`成员返回所有不需要优化的参数列表。如果希望获得所有参数列表，可以通过类的`variables`返回所有内部张量列表：

In [7]:
for v in fc.variables:
    print("name: ", v.name, ", shape: ", v.shape)

name:  dense/kernel:0 , shape:  (784, 512)
name:  dense/bias:0 , shape:  (512,)


利用网络层类对象进行前向计算时，只需要调用类的`__call__`方法即可，即写成`fc(x)`方式便可。对于全连接层类，在`__call__`方法中实现$\sigma(X@W + b)$的运算逻辑即可。

## 6.3 神经网络
通过层层堆叠全连接层，保证前一层的输出节点数与当前层的输入节点数匹配，即可堆叠出任意层数的网络。我们把这种由神经元相互连接而成的网络叫做神经网络。

<img src="images/06_05.png" style="width:500px;"/>

在设计全连接网络时，网络的结构配置等超参数可以按着经验法则自由设置，只需要遵循少量的约束即可。例如，隐藏层1的输入节点数需和数据的实际特征长度匹配，每层的输入层节点数与上一层输出节点数匹配，输出层的激活函数和节点数需要根据任务的具体设定进行设计。总的来说，神经网络模型的结构设计自由度较大，至于与哪一组超参数是最优的，这需要很多的领域经验知识和大量的实验尝试，或者可以通过`AutoML`技术搜索出较优设定。

### 6.3.1 张量方式实现
对于多层神经网络，需要分别定义各层的权值矩阵$W$和偏置向量$b$：

In [8]:
# 隐藏层1张量
w1 = tf.Variable(tf.random.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))
# 隐藏层2张量
w2 = tf.Variable(tf.random.truncated_normal([256, 128], stddev=0.1))
b2 = tf.Variable(tf.zeros([128]))
# 隐藏层3张量
w3 = tf.Variable(tf.random.truncated_normal([128, 64], stddev=0.1))
b3 = tf.Variable(tf.zeros([64]))
# 输出层张量
w4 = tf.Variable(tf.random.truncated_normal([64, 10], stddev=0.1))
b4 = tf.Variable(tf.zeros([10]))

在计算时，只需要按照网络层的顺序，将上一层的输出作为当前层的输入即可，重复直至最后一层，并将输出层的输出作为网络的输出：

In [9]:
with tf.GradientTape() as tape: # 梯度记录器
    # x: [b, 28*28]
    # 隐藏层1前向计算，[b, 28*28] => [b, 256]
    h1 = x@w1 + tf.broadcast_to(b1, [x.shape[0], 256])
    h1 = tf.nn.relu(h1)
    # 隐藏层2前向计算，[b, 256] => [b, 128]
    h2 = h1@w2 + b2
    h2 = tf.nn.relu(h2)
    # 隐藏层3前向计算，[b, 128] => [b, 64]
    h3 = h2@w3 + b3
    h3 = tf.nn.relu(h3)
    # 输出层前向计算，[b, 64] => [b, 10]
    h4 = h3@w4 + b4

最后一层是否需要添加激活函数通常视具体的任务而定，这里加不加都可以。

在使用自动求导功能计算梯度时，需要将前向计算过程放置在`tf.GradientTape()`环境中，从而利用`GradientTape`对象的`gradient()`方法自动求解参数的梯度，并利用`optimizers`对象更新参数。

### 6.3.2 层方式实现
对于常规的网络层，通过层方式实现起来更加简洁高效：

In [10]:
fc1 = layers.Dense(256, activation=tf.nn.relu) #隐藏层 1
fc2 = layers.Dense(128, activation=tf.nn.relu) #隐藏层 2
fc3 = layers.Dense(64, activation=tf.nn.relu) #隐藏层 3
fc4 = layers.Dense(10, activation=None) #输出层

x = tf.random.normal([4,28*28])
h1 = fc1(x)
h2 = fc2(h1)
h3 = fc3(h2)
h4 = fc4(h3)

对于这种数据依次向前传播的网络，也可以通过`Sequential`容器封装成一个网络大类对象，调用大类的前向计算函数一次即可完成所有层的前向计算：

In [11]:
# 通过Sequential容器封装为一个网络类
model = Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu), 
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(10, activation=None),
])
out = model(x)
out

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[-0.05488776,  0.15282597,  0.73966974, -0.51127696, -0.26608175,
         0.53888005, -0.7096812 , -0.12645325,  0.3285418 , -0.3526814 ],
       [-0.20345806,  0.22128472, -0.25527686, -0.17398459, -0.8212601 ,
         0.746904  , -0.47945023, -1.158399  , -0.2600506 , -0.3165411 ],
       [ 0.75445557,  0.46598294,  0.27885482, -1.2451789 , -0.2434003 ,
         0.74253947, -0.45031756,  0.3654445 ,  0.06243852, -0.51458883],
       [ 0.9610448 , -0.06296299, -0.2092366 , -0.6370686 ,  0.16406512,
         0.3134638 , -0.23745295, -0.33416578, -0.00441459, -0.2422839 ]],
      dtype=float32)>

### 6.3.3 优化目标
我们把神经网络从输入到输出的计算过程叫做`前向传播`(Forward Propagation)或前向计算。前向传播过程，也是张量(Tensor)从第一层流动(Flow)至输出层的过程，即从输入数据开始，途径每个隐藏层，直至得到输出并计算误差，这也是`TensorFlow`框架名字由来。

前向传播的最后一步就是完成误差的计算：
+ $\mathcal{L} = g(f_\theta(x), y)$

其中$f_\theta(\cdot)$代表了利用$\theta$参数化的神经网络模型，$g(\cdot)$称之为`误差函数`，用来描述当前网络的预测值与真实标签$y$之间的差距度量，比如`均方差误差函数`。$\mathcal{L}$称为网络的`误差`(Error，或`损失`Loss)，一般为标量。我们希望通过在训练集$\mathbb{D}^{\mathrm{train}}$上面学习到一组参数$\theta$使得训练的误差$\mathcal{L}$最小：
+ $\theta^{*} = \underbrace{\arg\min}_{\ \theta} g(f_\theta(x), y), \ x \in \mathbb{D}^{\mathrm{train}}$

上述的最小化优化问题一般采用`误差反向传播`(Backward Propagation，简称BP)算法来求解网络参数$\theta$的梯度信息，并利用`梯度下降`(Gradient Descent，简称GD)算法迭代更新参数：
+ $\theta' = \theta-\mu\cdot\nabla_\theta\mathcal{L}$

其中，$\mu$为学习率。 

从另一个角度来理解神经网络，它完成的是特征的维度变换的功能，比如4层的`MNIST`手写数字图片识别的全连接网络，它依次完成了$784 \to 256 \to 128 \to 64 \to 10$的特征降维过程。原始的特征通常具有较高的维度，包含了很多底层特征及无用信息，通过神经网络的层层特征变换，将较高的维度降维到较低的维度，此时的特征一般包含了与任务强相关的高层抽象特征信息，通过对这些特征进行简单的逻辑判定即可完成特定的任务，如图片的分类。

## 6.4 激活函数
本节介绍神经网络中的常见激活函数，与阶跃函数和符号函数不同，这些函数都是平滑可导的，适合于梯度下降算法。

### 6.4.1 Sigmoid
`Sigmoid`函数也叫`Logistic`函数，定义为
+ $sigmoid(x) = \displaystyle\frac{1}{1+e^{-x}}$

它的一个优良特性就是能够把$x \in \mathcal{R}$的输入压缩到$x \in (0,1)$区间，这个区间的数值在机器学习常用来表示以下意义：
+ 概率分布`(0,1)`区间的输出和概率的分布范围`[0,1]`契合，可以将`Sigmoid`函数的输出转译为概率输出
+ 信号强度：可以将`0~1`理解为某种信号的强度，如像素的颜色强度，1代表当前通道颜色最强，0代表当前通道无颜色；抑或代表`门控值`(Gate)的强度，1代表当前门控全部开放，0代表门控关闭

`Sigmoid函数`连续可导，如`图6.7`所示：

<img src="images/06_07.png" style="width:300px;"/>

通过`tf.nn.sigmoid`实现`Sigmoid`函数：

In [12]:
x = tf.linspace(-6.,6.,10)
tf.nn.sigmoid(x) # 通过Sigmoid函数

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.00247262, 0.00931596, 0.0344452 , 0.11920292, 0.33924365,
       0.6607564 , 0.880797  , 0.96555483, 0.99068403, 0.9975274 ],
      dtype=float32)>

### 6.4.2 ReLU
`Sigmoid`函数在输入值较大或较小时容易出现梯度值接近于0的现象，称为`梯度弥散现象`。出现梯度弥散现象时，网络参数长时间得不到更新，导致训练不收敛或停滞不动的现象发生，较深层次的网络模型中更容易出现梯度弥散现象。

`ReLU`(REctified Linear Unit，修正线性单元)是应用最广泛的激活函数之一，定义如下：
+ $\mathcal{ReLU} = \max(0,x)$

函数曲线如`图6.8`所示。`ReLU`对小于0的值全部抑制为0；对于正数则直接输出，这种单边抑制特性来源于生物学。

<img src="images/06_08.png" style="width:300px;"/>

通过`tf.nn.relu`实现`ReLU`函数：

In [13]:
tf.nn.relu(x) # 通过ReLU激活函数

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.      , 0.      , 0.      , 0.      , 0.      , 0.666667,
       2.      , 3.333334, 4.666667, 6.      ], dtype=float32)>

除了可以使用函数式接口`tf.nn.relu`实现`ReLU`函数外，还可以像`Dense`层一样将`ReLU`函数作为一个网络层添加到网络中，对应的类为`layers.ReLU()`类。一般来说，激活函数类并不是主要的网络运算层，不计入网络的层数。

ReLU 函数的设计源自神经科学，函数值和导数值的计算均十分简单，同时有着优良 的梯度特性，在大量的深度学习应用中被验证非常有效，是应用最广泛的激活函数之一。

### 6.4.3 LeakyReLU
`ReLU`函数在$x \lt 0$时导数值恒为0，也可能会造成梯度弥散现象，为了克服这个问题，`LeakyReLU`函数被提出，如`图6.10`所示:

<img src="images/06_10.png" style="width:300px;"/>

`LeakyReLU`的表达式为
+ $\mathcal{LeakyReLU} = \begin{cases} x,  & \text{x \ge 0} \\px, & \text{x \lt 0} \end{cases}$

其中$p$为用户自行设置的某较小数值的超参数，如0.02等。当$p = 0$时，`LeayReLU`函数退化为`ReLU`函数；当$p \ne 0$时，$x \lt 0$处能够获得较小的导数值$p$，从而避免出现梯度弥散现象。

通过`tf.nn.leaky_relu`实现`LeakyReLU`函数：

In [14]:
tf.nn.leaky_relu(x, alpha=0.01) # 通过 LeakyReLU 激活函数

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.06      , -0.04666666, -0.03333333, -0.02      , -0.00666666,
        0.666667  ,  2.        ,  3.333334  ,  4.666667  ,  6.        ],
      dtype=float32)>

`alpha`参数代表$p$。`tf.nn.leaky_relu`对应的类为`layers.LeakyReLU`，可以像`Dense`层一样将`LeakyReLU`层放置在网络的合适位置。

### 6.4.4 Tanh
`Tanh`函数能够将$x \in \mathcal{𝑅}$的输入压缩到`(−1,1)`区间，定义为：
+ $\tanh(x) = \displaystyle\frac{(e^x-e^{-x})}{(e^x+e^{-x})} = 2 \cdot \text{sigmoid}(2x)−1$

可以看到`tanh`激活函数可通过`Sigmoid`函数缩放平移后实现，函数曲线如`图6.11`：

<img src="images/06_10.png" style="width:300px;"/>

通过`tf.nn.tanh`实现`tanh`函数：

In [15]:
tf.nn.tanh(x) # 通过tanh激活函数

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.99998784, -0.99982315, -0.9974579 , -0.9640276 , -0.58278286,
        0.58278316,  0.9640276 ,  0.99745804,  0.99982315,  0.99998784],
      dtype=float32)>

## 6.5 输出层设计
网络的最后一层除了和所有的隐藏层一样完成维度变换、特征提取的功能外，还作为输出层使用，需要根据具体的任务场景来决定是否使用激活函数、以及使用什么类型的激活函数等。

我们将根据输出值的区间范围来分类讨论。常见的几种输出类型包括：
+ $o_i \in \mathcal{R}^d$输出属于整个实数空间，或者某段普通的实数空间，比如函数值趋势的预测，年龄的预测问题等
+ $o_i \in [0,1]$输出值特别地落在`[0, 1]`的区间，如图片像素值一般用`[0, 1]`区间的值表示；或者二分类问题的概率，如硬币正反面的概率预测问题
+ $o_i \in [0,1], \ \sum_{i}o_i=1$输出值落在`[0,1]`的区间，并且所有输出值之和为1，常见的如多分类问题，如`MNIST`手写数字图片识别，图片属于10个类别的概率之和应为1
+ $o_i \in [−1, 1]$输出值在`[-1, 1]`之间

### 6.5.1 普通实数空间
年龄的预测、股票走势的预测等都属于整个或者部分连续的实数空间，输出层可以不加激活函数。误差的计算直接基于最后一层的输出$o$和真实值$y$进行计算，如采用均方差误差函数度量输出值$o$与真实值$y$之间的距离：
+ $\mathcal{L} = g(o,y)$

其中$g$代表了某个具体的误差计算函数，例如`MSE`等。

### 6.5.2 [0, 1]区间
输出值属于`[0,1]`区间也比较常见，比如图片的生成、二分类问题等。

在机器学习中，一般会将图片的像素值归一化到`[0,1]`区间。为了让像素的值范围映射到`[0,1]`的有效实数空间，需要在输出层后添加某个合适的激活函数，其中`Sigmoid`函数刚好具有此功能。

对于二分类问题，如硬币的正反面的预测，输出层可以只设置一个节点，表示某个事件$A$发生的概率$P(A|x)$，$x$为网络输入。

### 6.5.3 [0,1]区间，和为1
这种设定以多分类问题最为常见。

如`图6.15`所示，输出层的每个输出节点代表了一种类别，3个节点的输出值分布代表了当前样本属于类别$A$、类别$B$和类别$C$的概率，考虑多分类问题中的样本只可能属于所有类别中的某一种，因此满足所有类别概率之和为1的约束。

<img src="images/06_15.png" style="width:450px;"/>

通过在输出层添加`Softmax`函数实现此约束逻辑。`Softmax`函数定义为
+ $\mathcal{Softmax}(z_i) = \displaystyle\frac{e^{z_i}}{\sum_{j=1}^{d_{out}}e^{z_j}}$

<img src="images/06_14.png" style="width:300px;"/>

通过`tf.nn.softmax`实现`Softmax`函数：

In [16]:
z = tf.constant([2.,1.,0.1])
tf.nn.softmax(z) # 通过Softmax函数

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.6590012 , 0.24243298, 0.09856589], dtype=float32)>

`Softmax`函数也可以作为网络层类使用，通过类`layers.Softmax(axis=-1)`可以方便添加`Softmax`层，其中`axis`参数指定需要进行计算的维度。

在`Softmax`函数的数值计算过程中，容易因输入值偏大发生数值溢出现象；在计算交叉熵时，也会出现数值溢出的问题。为了数值计算的稳定性，`TensorFlow`提供了一个统一的接口，将`Softmax`与`交叉熵损失函数`同时实现，同时也处理了数值不稳定的异常，一般推荐使用这些接口函数，避免分开使用`Softmax`函数与`交叉熵损失函数`。函数式接口为`tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False)`，其中`y_true`代表了`onehot`编码后的真实标签，`y_pred`表示网络的预测值，当`from_logits`设置为`True`时，`y_pred`表示须为未经过`Softmax`函数的变量`z`；当`from_logits`设置为`False`时，`y_pred`表示为经过`Softmax`函数的输出。为了数值计算稳定性，一般设置`from_logits`为`True`，此时`tf.keras.losses.categorical_crossentropy`将在内部进行`Softmax`函数计算，所以不需要在模型中显式调用`Softmax`函数。

In [17]:
z = tf.random.normal([2,10]) # 构造输出层的输出
y_onehot = tf.constant([1,3]) # 构造真实值
y_onehot = tf.one_hot(y_onehot, depth=10) #onehot编码
# 输出层未使用Softmax函数，故from_logits设置为True
# 这样categorical_crossentropy函数在计算损失函数前，会先内部调用Softmax函数
loss = keras.losses.categorical_crossentropy(y_onehot,z,from_logits=True)
loss = tf.reduce_mean(loss) # 计算平均交叉熵损失
loss

<tf.Tensor: shape=(), dtype=float32, numpy=2.2180057>

除了函数式接口，也可以利用`losses.CategoricalCrossentropy(from_logits)`类方式同时实现`Softmax`与交叉熵损失函数的计算，`from_logits`参数的设置方式相同。例如：

In [18]:
# 创建Softmax与交叉熵计算类，输出层的输出z未使用softmax
criteon = keras.losses.CategoricalCrossentropy(from_logits=True)
loss = criteon(y_onehot,z) # 计算损失
loss

<tf.Tensor: shape=(), dtype=float32, numpy=2.2180057>

### 6.5.4 [-1, 1]
如果希望输出值的范围分布在`(−1,1)`区间，可以简单地使用`tanh`激活函数：

In [19]:
x = tf.linspace(-6.,6.,10)
tf.tanh(x) # tanh 激活函数

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.99998784, -0.99982315, -0.9974579 , -0.9640276 , -0.58278286,
        0.58278316,  0.9640276 ,  0.99745804,  0.99982315,  0.99998784],
      dtype=float32)>

## 6.6 误差计算
在搭建完模型结构后，下一步就是选择合适的误差函数来计算误差。常见的误差函数有`均方差`、`交叉熵`、`KL散度`、`Hinge Loss函数`等，其中`均方差函数`和`交叉熵函数`在深度学习中比较常见，`均方差函数`主要用于回归问题，`交叉熵函数`主要用于分类问题。

### 6.6.1 均方差误差函数
`均方差`(Mean Squared Error，简称MSE)误差函数把输出向量和真实向量映射到笛卡尔坐标系的两个点上，通过计算这两个点之间的欧式距离的平方来衡量两个向量之间的差距：
+ $\mathrm{MSE} = \displaystyle\frac{1}{d_{\mathrm{out}}}\sum_{i=1}^{d_{\mathrm{out}}}(y_i-o_i)^2$

`MSE`误差函数的值总是大于等于0，当`MSE`函数达到最小值0时，输出等于真实标签。

均方差误差函数广泛应用在回归问题中，实际上，分类问题中也可以应用均方差误差函数。

可以通过函数方式或层方式实现`MSE`误差计算：

In [20]:
o = tf.random.normal([2,10]) # 构造网络输出
y_onehot = tf.constant([1,3]) # 构造真实值
y_onehot = tf.one_hot(y_onehot, depth=10)
loss = keras.losses.MSE(y_onehot, o) # 计算均方差
loss

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.69925654, 1.2358468 ], dtype=float32)>

`MSE`函数返回的是每个样本的均方差，需要得平均样本的均方差：

In [21]:
loss = tf.reduce_mean(loss) # 计算batch均方差
loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.96755165>

也可以通过层方式实现，对应的类为`keras.losses.MeanSquaredError()`，调用`__call__`函数即可完成前向计算：

In [22]:
criteon = keras.losses.MeanSquaredError()
loss = criteon(y_onehot,o) # 计算batch均方差
loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.96755165>

### 6.6.2 交叉熵误差函数
##### 熵
1948年，`Claude Shannon`将热力学中熵的概念引入到信息论中，用来衡量信息的不确定度。熵在信息学科中也叫信息熵，或者香农熵。熵越大，代表不确定性越大，信息量也就越大。

某个分布$P(i)$的熵定义为
+ $\mathcal{H}(P) = -\displaystyle\sum_{i}P(i)\log_2P(i)$

实际上，$\mathcal{H}(P)$也可以使用其他底数的$\log$函数计算。举个例子，对于4分类问题，如果某个样本的真实标签是第4类，那么标签的`onehot`编码为`[0,0,0,1]`，即这张图片的分类是唯一确定的，它属于第4类的概率$P(y为4|x) = 1$，不确定性为0，它的熵可以简单的计算为：
+ $− 0\cdot \log_2(0) − 0\cdot \log_2(0) − 0\cdot \log_2(0) − 1 \cdot \log_2(1) = 0$

也就是说，对于确定的分布，熵为0，不确定性最低。

如果它预测的概率分布是`[0.1,0.1,0.1,0.7]`，它的熵可以计算为：
+ $−0.1 \cdot \log_2(0.1) − 0.1 \cdot \log_2(0.1) − 0.1 \cdot \log_2(0.1) − 0.7 \cdot \log_2(0.7) ≈ 1.356$

考虑随机分类器，它每个类别的预测概率是均等的：`[0.25,0.25,0.25,0.25]`，熵约为2，这种情况的不确定性略大于上面一种情况。

由于$P(i) \in [0,1], \log_2(P(i)) \lt 0$，因此熵$\mathcal{H}(P)$总是大于等于0。当熵取得最小值0时，不确定性为0。

##### 交叉熵
> 参考[文章](https://blog.csdn.net/tsyccnh/article/details/79163834)

我们基于熵引出`交叉熵`(Cross Entropy)的定义：
+ $\mathcal{H}(p||q) = − \displaystyle\sum_{i}p(i)\log_2(q(i))$

通过变换，交叉熵可以分解为$p$的熵$\mathcal{H}(P)$和$p$与$q$的`KL散度`(Kullback-Leibler Divergence)的和：
+ $\mathcal{H}(p||q) = \mathcal{H}(p) + D_{KL}(p||q)$

其中`KL`定义为
+ $D_{KL}(p||q) = \displaystyle\sum_{i}p(i)\log\big(\frac{p(i)}{q(i)}\big )$

`KL散度`是`Solomon Kullback`和`Richard A. Leibler`在1951年提出的用于衡量2个分布之间距离的指标。$p = q$时，$D_{KL}(p||q)$取得最小值0，$p$与$q$之间的差距越大，$D_{KL}(p||q)$也越 大。需要注意的是，`交叉熵`和`KL散度`都不是对称的，即：
+ $\mathcal{H}(p||q) \ne \mathcal{H}(q||p)$
+ $D_{KL}(p||q) \ne D_{KL}(q||p)$

交叉熵可以很好地衡量2个分布之间的“距离”。特别地，当分类问题中$\mathrm{y}$的编码分布$p$采用`onehot`编码$y$时：$\mathcal{H}(p) = 0$，此时
+ $\mathcal{H}(p||q) = \mathcal{H}(p) + D_{KL}(p||q) = D_{KL}(p||q)$

退化到真实标签分布$y$与输出概率分布$o$之间的`KL散度`上。根据`KL散度`的定义，我们推导分类问题中交叉熵的计算表达式：
+ $\mathcal{H}(p||q) = D_{KL}(p||q) = \displaystyle\sum_{j}y_j\log(\frac{y_j}{o_j}) = 1\cdot \log(\frac{1}{o_i}) + \sum_{j\ne i} 0\cdot \log(\frac{0}{o_j}) = -\log(o_j)$

其中$i$为`onehot`编码中为1的索引号，也是当前输入的真实类别。可以看到，$\mathcal{L}$只与真实类别$i$上的概率$o_i$有关，对应概率$o_i$越大，$\mathcal{H}(p||q)$越小。当对应类别上的概率为1时，交叉熵$\mathcal{H}(p||q)取得最小值0，此时网络输出$o$与真实标签$y$完全一致，神经网络取得最优状态。

因此最小化交叉熵损失函数的过程也是最大化正确类别的预测概率的过程。从这个角度去理解交叉熵损失函数，非常地直观易懂。

## 6.7 神经网络类型
### 6.7.1 卷积神经网络
卷积神经网络(Convolutional Neural Network，简称CNN)在计算机视觉中的表现大大地超越了其它算法模型。这其中比较流行的模型有用于图片分类的`AlexNet`、`VGG`、`GoogLeNet`、`ResNet`、`DenseNet`等，用于目标识别的`RCNN`、`Fast RCNN`、`Faster RCNN`、`Mask RCNN`、`YOLO`、`SSD`等。

### 6.7.2 循环神经网络
循环神经网络(Recurrent Neural Network，简称RNN)非常擅长处理序列信号。1997年，`Jürgen Schmidhuber`提出的`LSTM`网络较好地克服了`RNN`缺乏长期记忆、不擅长处理长序列的问题，在自然语言处理中得到了广泛的应用。基于`LSTM`模型，`Google`提出了用于机器翻译的`Seq2Seq`模型，并成功商用于谷歌神经机器翻译系统(GNMT)。其他的`RNN`变种还有`GRU`、`双向RNN`等。

### 6.7.3 注意力(机制)网络
`注意力机制`(Attention Mechanism)克服了`RNN`训练不稳定、难以并行化等缺陷，在自然语言处理和图片生成等领域中逐渐崭露头角。

2017年，`Google`提出了第一个利用纯注意力机制实现的网络模型`Transformer`，随后基于`Transformer`模型相继提出了一系列的用于机器翻译的注意力网络模型，如`GPT`、`BERT`、`GPT-2`等。在其它领域，基于注意力机制，尤其是`自注意力`(SelfAttention)机制构建的网络也取得了不错的效果，比如基于自注意力机制的`BigGAN`模型等。

### 6.7.4 图卷积神经网络
图片、文本等数据具有规则的空间、时间结构，称为`Euclidean Data`(欧几里德数据)。卷积神经网络和循环神经网络被证明非常擅长处理这种类型的数据。而像类似于社交网络、通信网络、蛋白质分子结构等一系列的不规则空间拓扑结构的数据，它们显得力不从心。

2016年，`Thomas Kipf`等人提出了`图卷积网络`(Graph Convolution Network，GCN)模型。`GCN`算法实现简单，从空间一阶邻居信息聚合的角度也能直观地理解，在半监督任务上取得了不错效果。随后，一系列的网络模型相继被提出，如`GAT`，`EdgeConv`，`DeepGCN`等。